# Word Embeddings

In this part of the assignment, we'll explore a few properties of word embeddings. We'll use pre-trained GloVe ([Pennington et al. 2013](https://nlp.stanford.edu/pubs/glove.pdf)) embeddings, and evaluate on the analogy task described in ([Mikolov et al. 2013](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)).

If you haven't seen the [embeddings.ipynb](../../materials/embeddings/embeddings.ipynb) demo notebook, we recommend you look through it; this part of the assignment will build on that material.

In [2]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("wget")      # for fetching dataset

In [3]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# NLTK for NLP utils
import nltk

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz

# Fits like a GloVe

Word embeddings take a long time to train - since the goal is to provide a good representation for as many words as possible, generating good embeddings often requires making several passes over a very large corpus. 

Fortunately, it's possible to learn fairly general embeddings from large corpora that are useful for many downstream tasks. We'll use the GloVe vectors available at https://nlp.stanford.edu/projects/glove/ - specifically, a set trained with a vocabulary of 400,000 on a corpus of 6B tokens from Wikipedia and Gigaword.

The vectors are distributed as a (very) large text file, with one word per line followed by its vector:
```
the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459
```

We've implemented a helper class, `Hands` in `glove_helper.py`, that will parse these files in a memory efficient manner and provide a wrapper object over a NumPy array containing the actual vectors. 

Run the cell below; the first time, it will download an ~800 MB file to the `data/` directory. **_Please do not check this in to git!_**

In [4]:
import glove_helper; reload(glove_helper)

hands = glove_helper.Hands(ndim=100)  # 50, 100, 200, 300 dim are available

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.100d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 100))


`hands` has a few properties and methods that might be useful:
- `hands.vocab` is a `vocabulary.Vocabulary` object that manages the set of available words
- `hands.W` is a matrix of shape $|V| \times d$ containing the actual vectors, one per row. Row indices are as given by `hands.vocab.word_to_id[word]`.
- `hands.get_vector(word)` returns the vector for a word (passed as a string).

Note that we let $|V| = $`hands.W.shape[0]`, which in addition to the actual words includes three special tokens: `<s>` (begin sentence), `</s>` (end sentence), and `<unk>` (unknown word).

# Part (a): Nearest Neighbors

### Cosine Similarity

To measure the similarity of two words, we'll use the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) between their representation vectors:

$$ D^{cos}_{ij} = \frac{v_i^T v_j}{||v_i||\ ||v_j||}$$

*Note that this is called cosine similarity because $D^{cos}_{ij} = \cos(\theta_{ij})$, where $\theta_{ij}$ is the angle between the two vectors.*

## Part (a) Questions

1. In `vector_math.py`, implement the `find_nn_cos(...)` function. Read the docstring _carefully_ - it describes what you should return. *Hint: use NumPy functions instead of a `for` loop.*
<p>
2. Use the `show_nns(...)` function below to find the nearest neighbors for the words `"bank"`, `"plane"`, and `"flies"`. Put the closest word (other than *the exact word* itself) in the answers file.  If you inspect `"flies"`, for instance, are the neighbors dominated by one sense of the word or is there evidence that the vector encodes meaning of the other senses as well?
<p>
3. Like `word2vec`, GloVe constructs representations by summarizing word-word coocurrence statistics. Use `show_nns(...)` to find the neighbors of `"green"` and `"celadon"`, and `"orange"`. Explain what you find in terms of the distributional hypothesis and the grounding problem. Does the vector for `"celadon"` appear to encode a notion of color? What do they represent, instead?

_(The Distributional Hypothesis is the idea that "you shall know a word by the company it keeps" (Firth, 1957) - that meaning is derived from the context in which a word is used. Grounding refers to the meaning of language in terms of external concepts, such as real-world entities or physical characteristics.  Take a look at the readings from the first week if you're not confident.)_


In [36]:
import vector_math; reload(vector_math)

def show_nns(hands, word, k=10):
    """Helper function to print neighbors of a given word."""
    word = word.lower()
    print("Nearest neighbors for '{:s}'".format(word))
    v = hands.get_vector(word)
    for i, sim in zip(*vector_math.find_nn_cos(v, hands.W, k)):
        target_word = hands.vocab.id_to_word[i]
        print("{:.03f} : '{:s}'".format(sim, target_word))
    print("")
    
show_nns(hands, "the")

Nearest neighbors for 'the'
33.886 : 'the'
29.051 : 'this'
28.830 : 'part'
28.813 : 'one'
28.223 : 'of'
28.208 : 'same'
27.822 : 'first'
27.786 : 'on'
27.683 : 'its'
27.544 : 'as'



In [38]:
#### YOUR CODE HERE ####
# Code for Part (a).2

words_of_interest = ["bank","plane","flies"]

for word in words_of_interest:
    show_nns(hands, word)
#### END(YOUR CODE) ####

Nearest neighbors for 'bank'
46.400 : 'bank'
37.385 : 'banks'
34.943 : 'banking'
32.655 : 'credit'
32.202 : 'investment'
31.447 : 'financial'
31.034 : 'securities'
30.833 : 'lending'
30.090 : 'funds'
30.083 : 'ubs'

Nearest neighbors for 'plane'
40.725 : 'plane'
35.235 : 'airplane'
33.458 : 'flight'
33.378 : 'jet'
32.898 : 'crashed'
32.288 : 'crash'
31.563 : 'planes'
31.479 : 'helicopter'
30.859 : 'airliner'
30.678 : 'flying'

Nearest neighbors for 'flies'
26.477 : 'flies'
19.627 : 'fly'
17.045 : 'flying'
16.779 : 'insects'
16.745 : 'flew'
16.355 : 'butterflies'
16.259 : 'moths'
16.134 : 'moth'
15.396 : 'planes'
15.253 : 'plane'



In [39]:
#### YOUR CODE HERE ####
# Code for Part (a).3
words_of_interest = ["green","celadon","orange"]

for word in words_of_interest:
    show_nns(hands, word)

#### END(YOUR CODE) ####

Nearest neighbors for 'green'
33.239 : 'green'
27.272 : 'red'
26.153 : 'blue'
25.955 : 'brown'
25.612 : 'yellow'
25.342 : 'white'
24.899 : 'gray'
24.367 : 'black'
24.230 : 'pink'
24.187 : 'purple'

Nearest neighbors for 'celadon'
18.135 : 'celadon'
11.238 : 'faience'
10.926 : 'porcelains'
10.780 : 'majolica'
10.725 : 'ocher'
10.614 : 'blue-and-white'
10.424 : 'glazes'
10.207 : 'unglazed'
10.118 : 'porcelain'
9.952 : 'steatite'

Nearest neighbors for 'orange'
30.715 : 'orange'
22.602 : 'yellow'
21.933 : 'red'
21.863 : 'blue'
21.843 : 'green'
20.810 : 'pink'
20.807 : 'purple'
20.609 : 'black'
20.433 : 'colored'
19.203 : 'lemon'



# Part (b): Linear Analogies

In this part, you'll implement the word analogy task described in Section 4 of ([Mikolov et al. 2013](https://arxiv.org/pdf/1301.3781.pdf)), and discussed in section 4.8 and 4.11 of the async.

1. In `vector_math.py`, implement the `analogy(...)` function. (*Hint: this should be a very short function, given what you've already written above.*)
<p>

2. Evaluate the following analogies:
<ul>
<li> `"mouse" is to "mice" as "dog" is to ____`
<li> `"fast" is to  "fastest"   as "slow" is to ____`
<li> `"work" is to  "works"   as "speak" is to ____`
<li> `"russia" is to  "moscow"   as "greece" is to ____`
</ul>

<p>

3. (Ungraded) Play with a few analogies using the `show_analogy(...)` function below. Concretely, see if you can find at least one more analogy that tests each of the following relationships, and that the model gets right:<ul>
<li> Singular / plural
<li> Superlatives
<li> Verb tense
<li> Country / capital
</ul>

(See Table 1 of [Mikolov et al. 2013](https://arxiv.org/pdf/1301.3781.pdf) for a few ideas)

<p>

4. Evaluate the following analogies:
<ul>
<li> `"lizard" is to "reptile" as "dog" is to ____`
<li> `"finger" is to  "hand"   as "toe" is to ____`
</ul>


Similar to before, report the closest word (highest scoring) and exclude any exact match to any of the three words in the analogy query.

Think about what types of relations do these test? Does our approach of linear analogies work well here? What assumption is violated by these sorts of relationships? (_Hint: what if we reversed the order, and tested "reptile" is to "lizard", and so on?_ See [Meronymy](https://en.wikipedia.org/wiki/Meronymy).)

In [40]:
import vector_math; reload(vector_math)

def show_analogy(hands, a, b, c, k=5):
    """Compute and print a vector analogy."""
    a, b, c = a.lower(), b.lower(), c.lower()
    va = hands.get_vector(a)
    vb = hands.get_vector(b)
    vc = hands.get_vector(c)
    print("'{a:s}' is to '{b:s}' as '{c:s}' is to ___".format(**locals()))
    for i, sim in zip(*vector_math.analogy(va, vb, vc, hands.W, k)):
        target_word = hands.vocab.id_to_word[i]
        print("{:.03f} : '{:s}'".format(sim, target_word))
    print("")
    
show_analogy(hands, "king", "queen", "man")

'king' is to 'queen' as 'man' is to ___
37.337 : 'woman'
36.183 : 'man'
34.131 : 'girl'
31.663 : 'she'
30.614 : 'her'



In [44]:
#### YOUR CODE HERE ####
# Code for Part (b).2

show_analogy(hands, "mouse", "mice", "dog")
show_analogy(hands, "fast", "fastest", "slow")
show_analogy(hands, "work", "works", "speak")
show_analogy(hands, "russia", "moscow", "greece")

#### END(YOUR CODE) ####

'mouse' is to 'mice' as 'dog' is to ___
35.748 : 'dogs'
32.258 : 'rats'
31.631 : 'dog'
31.167 : 'cats'
30.493 : 'mice'

'fast' is to 'fastest' as 'slow' is to ___
36.162 : 'fastest'
31.471 : 'slowest'
27.071 : 'fourth'
27.045 : 'slower'
26.593 : 'slow'

'work' is to 'works' as 'speak' is to ___
25.558 : 'speak'
22.500 : 'speaks'
21.599 : 'spoken'
18.836 : 'spoke'
18.651 : 'speaking'

'russia' is to 'moscow' as 'greece' is to ___
28.751 : 'athens'
27.165 : 'greece'
25.276 : 'thessaloniki'
25.175 : 'istanbul'
24.082 : 'moscow'



In [45]:
#### YOUR CODE HERE ####
# Code for Part (b).3
show_analogy(hands, "lion", "lions", "bird")
show_analogy(hands, "easy", "easiest", "hard")
show_analogy(hands, "run", "ran", "heat")
show_analogy(hands, "china", "beijing", "taiwan")
#### END(YOUR CODE) ####

'lion' is to 'lions' as 'bird' is to ___
39.463 : 'bird'
37.971 : 'birds'
36.365 : 'ducks'
35.651 : 'dolphins'
34.462 : 'avian'

'easy' is to 'easiest' as 'hard' is to ___
17.383 : 'easiest'
15.983 : 'hardest'
15.583 : 'toughest'
15.154 : 'hard'
14.477 : 'harder'

'run' is to 'ran' as 'heat' is to ___
40.257 : 'heat'
27.677 : 'temperature'
27.456 : 'hot'
26.176 : 'boiling'
25.608 : 'heated'

'china' is to 'beijing' as 'taiwan' is to ___
36.794 : 'taipei'
35.536 : 'taiwan'
34.908 : 'beijing'
31.429 : 'taiwanese'
31.271 : 'seoul'



In [42]:
#### YOUR CODE HERE ####
# Code for Part (b).4

show_analogy(hands, "lizard", "reptile", "dog")
show_analogy(hands, "finger", "hand", "toe")

#### END(YOUR CODE) ####

'lizard' is to 'reptile' as 'dog' is to ___
38.067 : 'dog'
33.797 : 'dogs'
32.794 : 'pet'
31.464 : 'puppy'
31.193 : 'animal'

'finger' is to 'hand' as 'toe' is to ___
31.098 : 'toe'
25.056 : 'hand'
23.316 : 'shoes'
22.737 : 'back'
22.680 : 'hands'

